In [1]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import plotly.express as px
import plotly.io as pio
from folium import IFrame

# Load data
df_sudo_major = pd.read_csv("csv/df_sudo.csv")

# Calculate Weekly Median Rent
df_sudo_major["Median rent weekly"] = df_sudo_major["Rent and Mortgage Payments - Census Average monthly household rental payment ($)"] / 4.34524

# Read latitude and longitude data
df_lat_long = pd.read_csv("csv/sydney_melbourne_extracted_sa4_info.csv")
SA4_COORDINATES = dict(zip(df_lat_long['sa4_code'], zip(df_lat_long['latitude'], df_lat_long['longitude'])))

# Merge df_sudo_major with df_lat_long on 'SA4 Code'
df_sudo_major = pd.merge(df_sudo_major, df_lat_long[['sa4_code', 'sa4_name']], left_on='SA4 Code', right_on='sa4_code', how='left')

# Map SA4 Code to coordinates
df_sudo_major['Latitude'] = df_sudo_major['SA4 Code'].map(lambda code: SA4_COORDINATES.get(code, [None, None])[0])
df_sudo_major['Longitude'] = df_sudo_major['SA4 Code'].map(lambda code: SA4_COORDINATES.get(code, [None, None])[1])

# Define the columns not all of them thpugh yet
columns_to_plot = [
    'Median rent weekly', 
    'House Median Sales Price', 
    'Attached dwellings Median Sales Price',
    'Rent and Mortgage Payments - Census Average monthly household mortgage payment ($)',
    'Household Stress - Census Households with rent payments greater than or equal to 30% of household income (%)',
    'Rent and Mortgage Payments - Census Average monthly household mortgage payment ($)',
    'Rent and Mortgage Payments - Census Average monthly household rental payment ($)',
    'Housing Suitability - Occupied private dwellings - Census Dwellings with extra bedrooms needed (no.)',
    'Household Stress - Census Households with mortgage repayments greater than or equal to 30% of household income (%)'
]

# Filter out rows with missing coordinates
df_sudo_major = df_sudo_major.dropna(subset=['Latitude', 'Longitude', columns_to_plot[0]])

# Create a base map centered on Australia
australia_center = [-25.2744, 133.7751]  # Latitude and longitude of the center of Australia
m = folium.Map(location=australia_center, zoom_start=4, tiles='OpenStreetMap')

# Function to create a line graph for a specific column
def create_line_graph(df, column, sa4_name):
    fig = px.line(df, x="Year", y=column, title=f'{column} for {sa4_name}')
    fig.update_traces(mode='lines+markers', marker=dict(size=10, color='rgb(50, 100, 150)'), line=dict(width=3, color='rgb(50, 100, 150)'))  # Adjust colors and width
    fig.update_layout(
        paper_bgcolor='rgba(0, 0, 0, 0)',
        xaxis_title="Year",
        yaxis_title=f"{column}",
    )
    return fig

# Create heatmaps and line graphs for each column
for column in columns_to_plot:
    heatmap_data = df_sudo_major[['Latitude', 'Longitude', column]].dropna().values.tolist()
    HeatMap(heatmap_data).add_to(m)

    for sa4_code, sa4_name in df_sudo_major[['SA4 Code', 'sa4_name']].drop_duplicates().values:
        df_sa4 = df_sudo_major[df_sudo_major['SA4 Code'] == sa4_code]
        fig = create_line_graph(df_sa4, column, sa4_name)

        html_str = pio.to_html(fig, full_html=True, include_plotlyjs='cdn')
        iframe = IFrame(html=html_str, width=500, height=300)
        popup = folium.Popup(iframe, max_width=500)

        lat, lon = SA4_COORDINATES[sa4_code]
        marker = folium.Marker([lat, lon], popup=popup)
        marker.add_to(m)

# Add the layer control to the map
folium.LayerControl(position='topright', collapsed=False).add_to(m)

# Save the map to an HTML file
map_filename = "output/plotly_test.html"
m.save(map_filename)

print(f"Heatmap has been saved to '{map_filename}'")


Heatmap has been saved to 'output/plotly_test.html'
